In [ ]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import anndata as ad
import spatialdata as sd
from spatialdata_io import xenium

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq


saveFolder="/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF"
FigOut="/projects/b1042/Yuanqing/Sptial/data/output"
folderPath="/projects/b1146/BharatLab/20240530__190352__20240530_X1_Bharat"
sampleFile=glob.glob(f'{folderPath}/*__*')
disease=["IPF4","IPF1", "IPF3L", "IPF2", "Donor1", "IPF5", "IPF3R", "Donor2"]
zarr=[saveFolder+f'/Zarr/{disease[i]}.zarr' for i in range(len(disease))]
pdata=pd.DataFrame({'sam':sampleFile, 'disease':disease, 'Zarr':zarr})
pdata.disease

disease=['Donor1','Donor2',"IPF4","IPF1", "IPF3L", "IPF2","IPF5", "IPF3R"]
readPath='/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF/Zarr_withCT'

In [ ]:
from matplotlib import rcParams
# Use a vector-friendly PDF backend
rcParams["pdf.fonttype"] = 42  # Ensures text is stored as text, not paths
rcParams["ps.fonttype"] = 42


In [ ]:
mergDat=sc.read_h5ad(f"{saveFolder}/anndata_IPF_Coo_CT_Final.h5ad")
mergDat

In [ ]:
cellIn=pd.DataFrame(list(mergDat.obs['CellType'].cat.categories))
cellIn=cellIn.rename(columns={cellIn.columns[0]:'CellType'})
cellIn.rename(columns={'CellType':'CellType'},inplace=True)
cellIn.head(5)

In [ ]:
colSch=["#E41A1C", "#B43547", "#845172", "#546C9D", "#3983AC", "#3F908E", "#FFB315", "#FFD723", "#FFFA31", "#E9D630",
             "#85658D", "#9B4F9D", "#B75D70", "#D46A43", "#F07816", "#FF9007",
             'blue','cyan','#00FF00','#008b00',##last one is green4####. B-CD8T   "#FFB315", "#FFD723", "#FFFA31", "#E9D630",
             "#D0A72D", "#B8782A", "#AB5832", "#C1645C",
             "#D87085", "#EE7CAF", "#E685B8", "#CC8BAD", "#B292A3", "#999999"]
cellLevel=['AT2','AT1','Ciliated','Club','Basal','KRT17+KRT5-',
           'En',"Capillary.En","Artery.En","Lym.En",
           "Fibro","AlvF","MyoF","SMC","Pericyte","Mesothelial",
           "B","Plasma","CD4T","CD8T","NK",
           "Monocyte","DC","Macro","MoM","IM","Neutrophils","Chemokine.Imm","Prolif.Cells","Mast"]

In [ ]:
cellcol=pd.DataFrame({"CellType": cellLevel,'colSch':colSch})
cellcol.head(5)

In [ ]:
left_cell = pd.merge(cellIn, cellcol, on="CellType", how="left")
left_cell.head(5)

In [ ]:
left_cell.colSch.head(5)

In [ ]:
#mergDat.obs['CellType']=pd.Categorical(mergDat.obs['CellType'],categories=cellLevel,ordered=True)
mergDat.uns['CellType_colors']=left_cell.colSch

In [ ]:
sq.pl.nhood_enrichment(mergDat, 
                       figsize=[6,10],
                       cluster_key="CellType",
                       vmin=0,  
                       vmax=40.0,
                       show=False)
pdf_path = f"{FigOut}/allDat_IPF_coo_ordered.pdf"
plt.savefig(pdf_path, format="pdf", bbox_inches="tight")